## Using API classes to get data and save to db

# <span style="color:red">clear all output before saving: db output contains passwords! </span>
 
- importing modules needed
- creates a temporary db using .env
- creates the API objects for each vendor/station type
- pulls data from the vendor API


In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

from ewxpwsdb.db.models import WeatherStation, APIResponse, Reading, StationType
from ewxpwsdb.db.importdata import import_station_file, read_station_table
station_file = '../data/test_stations.tsv'

### optional: create new temp database to work with

In [3]:

from ewxpwsdb.db.database import Session, init_db, get_db_url, get_engine
from sqlmodel import select, delete

db_url = get_db_url()
print(db_url)
engine = get_engine(db_url)
init_db(engine)

# import, using the global 'engine' defined in database module
import_station_file(station_file)

ERROR:root:error creating new database: 'options' is an invalid keyword argument for Connection()


sqlite:///ewxpws.db


TypeError: 'options' is an invalid keyword argument for Connection()

## Create WeatherStation object

In [ ]:
station_type = 'SPECTRUM'


In [ ]:

# conventience function get the first station from the database for a specific type
def get_one_station(station_type, engine = engine):
    with Session(engine) as session:
        statement = select(WeatherStation).where(WeatherStation.station_type == station_type)
        results = session.exec(statement)
        weather_station = results.first()

    return(weather_station)





### create station object from database

In [ ]:
station = get_one_station(station_type)
station

 *could create weather station object without database, but then there is no ID Field which then can't be used to create related objects*

In [ ]:
# stations = read_station_table(station_file)
# # add code to find the correct one
# station_data = list(filter(lambda x: (x['station_type']==station_type), stations))[0]
# station = WeatherStation.model_validate(station_data) 
# station.station_type


## Test APIs


In [ ]:
# create API class from station
from ewxpwsdb.weather_apis import API_CLASS_TYPES
wapi = API_CLASS_TYPES[station.station_type](station)
wapi.station_type

In [ ]:
print(wapi.station_type)
print(wapi.weather_station.id)
print(wapi.sampling_interval)
print(wapi.APIConfigClass)
# check that configuration class is instantiated with same data in database
api_config = wapi.APIConfigClass.model_validate_json_str(wapi.weather_station.api_config)
print(api_config == wapi.api_config)

Get the data from an API request, and save that API_response into the database

In [ ]:
api_response_records = wapi.get_readings()
# check that there is some weather data, and 200 status
api_response_records[0].response_text

save the api responses from the request in the database, which then assigns and ID number(s)

In [ ]:
session = Session(engine)
for response in api_response_records:
    session.add(response)
    session.commit()

# session is still open

check that the current records that are inside the weatherapi were assigned a database ID

In [ ]:

print(wapi.current_api_response_records[0].id)

transform/harmonize the response data into sensor values.  

In [ ]:
readings = wapi.transform(wapi.current_api_response_records)
readings

check that these data can be turned into a Reading object (data + metadata )

save the rows of data from the sensor into the database using a Session

In [ ]:
# readings from transform...
for reading in readings:
    session.add(reading)    
session.commit()
# does it have an id now?
readings[0].id    

In [ ]:
readings[0].id 

## Getting data back from the database

In [ ]:
# check we still have a station id. 
station.id

In [ ]:
# summarize readings in the database
station_id = station.id
with Session(engine) as session:
    stmt = select(Reading, WeatherStation).join(WeatherStation).where(WeatherStation.id  == station.id)
    # results = session.exec(stmt)
    reading_records = session.exec(stmt).all()
    for reading_records in reading_records:
        reading = reading_records.Reading
        print(f"{reading.data_datetime}: air temp {reading.atemp}C")

    # let's save one for later
    reading = readings[0]



In [ ]:
# get more data
# note that the 'session' must be present for the whole transaction of response and readings for
responses = wapi.get_readings()
session = Session(engine)
for response in responses:
    session.add(response)
    session.commit()

responses[0].id
    
responses[0]

In [ ]:
responses[0].id

In [ ]:
stmt = select(APIResponse).where(APIResponse.request_id == wapi.current_api_response_records[0].request_id)
result = session.exec(stmt)
some_apiresponse = result.first()
print(some_apiresponse.id)

In [ ]:
api_response_records = api_response_records or wapi.current_api_response_records
api_response_records[0].id

## Clean up 

If using databases, remove test databases

In [ ]:
# if sqlite
import re
if re.match('sqlite', get_db_url()):
    from os import remove
    remove('ewxpws.db')

In [ ]:
# if postgresl
import re
from sqlmodel import delete, text
drop_stmt = text("""drop database ewxpws""")

if re.match('postgres', get_db_url()):
    with Session(engine) as session:
        session.exec(delete(Reading))
        session.exec(delete(APIResponse))
        session.exec(delete(WeatherStation))
        session.exec(delete(StationType))

        session.commit()

    # con = engine.connect()
    # con.execute(drop_stmt)
    # con.close()


In [ ]:
session.close()
engine.dispose()